In [ ]:
import numpy as np
from torch import nn
import torch.distributed as dist
from torch.utils.data import DataLoader
from torch.nn.parallel.distributed import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
from torchinfo import summary

from dataset import ATMADataset
from models.timesformer_gru import TimesformerGRU

In [ ]:
model = TimesformerGRU(pretrained_tsf="facebook/timesformer-base-finetuned-k400",
                       gru_hidden_size=128, gru_layers=2,
                       num_classes=10)

In [ ]:
dummy_input = list(np.random.rand(8 ,3, 224, 224))
summary(model=model)

In [ ]:
dataset = ATMADataset(vid_folder_path="./datasets/ATMA-V/videos/train/split",
                    label_path="./datasets/ATMA-V/labels/labels_aug.txt")

train_sampler =
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)